# add gaussian noise to a `Target`

You may want to generate a target with noise on the parameters without having to run the whole dataset + fitting process. 

The `apply_gaussian_noise` method of the ``Target`` object is made for that.

### noise model follows the modeldag approach.

The errors are drawn using the [modeldag](https://modeldag.readthedocs.io/) structure: `{"name": {"func": , "kwargs":{}}, }`

This model specifies how the **parameter error** is drawn. Then, the "observed" parameter will be drawn from a normal distribution centered on the "true" parameter with the parameter error as width, so: ``param_obs = np.normal.rvs(loc=param_true, scale=param_err)``


In [1]:
import skysurvey
import numpy as np

***
## Most simple case, random noise on 1 parameter


### step 0: create a timeserie target based on the "v19-2005bf" sncosmo template.

In [7]:
target = skysurvey.TSTransient(source_or_template="v19-2005bf")
# specify the absolute magnitude of the target (nan by default)
target.set_magabs([-18, 1])

# draw 1000 target, use inplace=True to change self.data
_ = target.draw(1_000, inplace=True)
target.data.head(3)

,z,t0,magabs,ra,dec,magobs,amplitude,template
0,0.02505,56189.019531,-18.655567,115.120918,-76.724373,16.611847,1.240252e-14,v19-2005bf
1,0.04825,56112.398438,-17.194258,200.798187,-39.482414,19.533182,8.413479e-16,v19-2005bf
2,0.04545,56178.195312,-18.262510,43.040348,22.518963,18.330763,2.546497e-15,v19-2005bf


### step 1: create an errormodel for `magobs`

Let's draw error on `magobs` assuming they are $0.1 ± 0.01$:

In [8]:
import numpy as np
errormodel = {"magobs": {"func": np.random.normal, "kwargs":{"loc":0.1, "scale":0.01}} }

### step 2: apply gaussian noise to the target's parameter

You will have a warning message, because the code doesn't know who to draw the error. 

In [9]:
noisy_target = target.apply_gaussian_noise(errormodel)
noisy_target.data.head(1)

,z,t0,magabs,ra,dec,magobs_true,amplitude,template,magobs_err,magobs
0,0.02505,56189.019531,-18.655567,115.120918,-76.724373,16.611847,1.240252e-14,v19-2005bf,0.108406,16.551349


### what happened ?

for each parameter given in the errormodel (here, only `magobs`):

  1. rename `{parameter}` as `{parameter}_true`
  2. draw `{parameter}_{err}` following the errormodel
  4. draw `{parameter}` from a normal distribution centered on `{parameter}_true` with a width of `{parameter}_{err}`